In [ ]:
import os
import numpy as np
import mne
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from itertools import groupby
%matplotlib inline
import glob

#### Grand Average

In [ ]:
def get_grand_average(subjs, outfile, exclude=[]):
    standard_subjs = []
    neutral_subjs = []
    rise_subjs = []
    fall_subjs = []

    for i in range(len(exclude)):
        exclude[i] = f'./analysis/Rise00{exclude[i]}-ave.fif'

    subjs = list(set(subjs) - set(exclude))
    
    # Separate evoked objects of all subjects into different conditions
    for subj in subjs:
        standard_subjs.append(mne.read_evokeds(subj, condition='1001'))
        neutral_subjs.append(mne.read_evokeds(subj, condition='1002'))
        rise_subjs.append(mne.read_evokeds(subj, condition='1003'))
        fall_subjs.append(mne.read_evokeds(subj, condition='1004'))

    # Combine evoked objects in each condition
    standard_combined = mne.combine_evoked(standard_subjs, weights='nave')
    neutral_combined = mne.combine_evoked(neutral_subjs, weights='nave')
    rise_combined = mne.combine_evoked(rise_subjs, weights='nave')
    fall_combined = mne.combine_evoked(fall_subjs, weights='nave')

    # Save combined evoked objects of all conditions into file
    mne.write_evokeds(f'./analysis/{outfile}', [standard_combined, neutral_combined, rise_combined, fall_combined], overwrite=True)

    evokeds = dict(standard=standard_combined, neutral=neutral_combined, rise=rise_combined, fall=fall_combined)

    return evokeds

In [ ]:
def plot_grand_average(evokeds, outfile, channels=[]):
    # Create PDF file in which to save all plots
    with matplotlib.backends.backend_pdf.PdfPages(f'./analysis/{outfile}') as pdf:
    
        for channel in channels:
            fig = mne.viz.plot_compare_evokeds(evokeds, picks=channel, combine=None, time_unit='ms', ylim=dict(eeg=[-5, 5]), invert_y=True,
                                            colors=dict(standard='black', neutral='red', rise='blue', fall='green'), 
                                            styles={'standard': {'linewidth': 1}, 'neutral': {'linewidth': 1}, 'rise': {'linewidth': 1}, 'fall': {'linewidth': 1}})
            # Save plot to PDF
            pdf.savefig(fig[0])
            plt.close()

In [ ]:
# Get list of evoked objects of all subjects
all_subj_evoked = glob.glob('./analysis/Rise*-ave.fif')

# Get grand average of all subjects
evokeds = get_grand_average(all_subj_evoked, 'grand-ave.fif', exclude=[])
# Get grand average with subjects 22, 25, 33, 34 and 36 excluded
evokeds_excluded = get_grand_average(all_subj_evoked, 'excl-grand-ave.fif', exclude=['2', '3', '22', '25', '33', '34', '36'])

# Plot channels Fz, Pz, Oz, AFz, POz, CPz, FCz, Cz for all subjects
plot_grand_average(evokeds, 'grand-ave-plots.pdf', channels=['Fz', 'Pz', 'Oz', 'AFz', 'POz', 'CPz', 'FCz', 'Cz'])
# Plot channels Fz, Pz, Oz, AFz, POz, CPz, FCz, Cz for all subjects excluding 22, 25, 33, 34 and 36
plot_grand_average(evokeds_excluded, 'excl-grand-ave.fif', channels=['Fz', 'Pz', 'Oz', 'AFz', 'POz', 'CPz', 'FCz', 'Cz'])